In [1]:
%%writefile workspace.json
{
"subscription_id" : "XXX",
"resource_group" : "XXX",
"workspace_name" : "XXX"
}

Overwriting workspace.json


In [2]:
#create an MLClient object that can interact with an Azure Machine Learning workspace. The code tries to use the DefaultAzureCredential to authenticate with Azure
#The code then reads the workspace.json file, which contains the subscription ID, resource group name, and workspace name of the Azure ML workspace. It uses these values to create a Workspace object, which represents the Azure ML workspace.

from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential=credential, path="workspace.json")
except Exception as ex:
    raise Exception(
        "Failed to create MLClient from config file. Please modify and then run the above cell with your AzureML Workspace details."
    ) from ex
    # ml_client = MLClient(
    #     credential=credential,
    #     subscription_id="",
    #     resource_group_name="",
    #     workspace_name=""
    # )

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ml_client)

Found the config file in: workspace.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7faba9c23730>,
         subscription_id=fe38c376-b42a-4741-9e7c-f5d7c31e5873,
         resource_group_name=yelizkilinc-rg,
         workspace_name=aml-prod)


In [3]:
import uuid

AOAI_endpoint = "https://openaiykus.openai.azure.com/" 
key_value="9ed9191bbdae445cbe69af2709668995"

keyvault = ws.get_default_keyvault()
keyvault.set_secret(name="aoai-endpoint", value = AOAI_endpoint)
keyvault.set_secret(name="key",value=key_value)


In [4]:
aoai_endpoint=keyvault.get_secret(name="aoai-endpoint")
aoai_key=keyvault.get_secret(name="key")

In [5]:
import openai

In [6]:
# Set up Azure OpenAI
openai.api_type = "azure"
openai.api_base = aoai_endpoint # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
openai.api_version = "2022-12-01"
openai.api_key = aoai_key

In [7]:
prompt= "Extract the person name, company name, location and phone number from the text below.\n\n Hello. My name is Robert Smith. I’m calling from Contoso Insurance, Delaware. My colleague mentioned that you are interested in learning about our comprehensive benefits policy. Could you give me a call back at (555) 346-9322 when you get a chance so we can go over the benefits?"

In [8]:
response = openai.Completion.create(
  deployment_id="textdavinci003yk", 
  prompt= prompt,
  temperature=0.1,
  max_tokens=100,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)


In [9]:
# print response
response['choices'][0]['text']

'\n\nPerson Name: Robert Smith\nCompany Name: Contoso Insurance\nLocation: Delaware\nPhone Number: (555) 346-9322'